# 학생 규모별 강좌수 전처리 템플릿


In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd

# 파일 경로 예시
file_path = '2023년 _대학_12-나-1. 학생 규모별 강좌수_학교별자료.xlsx'

# 엑셀 파일 로드 (헤더 없음으로 가정)
df_raw = pd.read_excel(file_path, header=None)
df_raw.iloc[:10, :12]  # 앞부분 확인

In [ ]:
# 5행부터 유의미한 데이터 시작, 필요한 열만 추출
df = df_raw.iloc[5:, [0, 5, 6, 8, 9, 10, 11]].copy()
df.columns = ['연도', '학교', '학기', '20명 이하', '21~30명', '31~40명', '41~50명']
df[['연도', '학교']] = df[['연도', '학교']].ffill()

In [ ]:
# 수치형 변환 및 범주 병합
cols_orig = ['20명 이하', '21~30명', '31~40명', '41~50명']
df[cols_orig] = df[cols_orig].apply(pd.to_numeric, errors='coerce').fillna(0)

df['20명 미만'] = df['20명 이하']
df['21~50명 미만'] = df[['21~30명', '31~40명', '41~50명']].sum(axis=1)
df['51~100명 미만'] = 0
df['101~200명 미만'] = 0
df['200명 이상'] = 0

In [ ]:
# 파생 피처 생성
cols = ['20명 미만', '21~50명 미만', '51~100명 미만', '101~200명 미만', '200명 이상']
df['총 강좌 수'] = df[cols].sum(axis=1)
df['소규모 강좌 비율'] = df['20명 미만'] / df['총 강좌 수']
df['대형 강좌 비율'] = df['200명 이상'] / df['총 강좌 수']

weights = [10, 30, 75, 150, 250]
df['평균 강좌 규모 점수'] = (
    df[cols].mul(weights).sum(axis=1) / df['총 강좌 수']
).round(1)

In [ ]:
# 학교 + 연도 기준 요약
df_summary = df.groupby(['연도', '학교']).agg({
    '총 강좌 수': 'sum',
    '소규모 강좌 비율': 'mean',
    '대형 강좌 비율': 'mean',
    '평균 강좌 규모 점수': 'mean'
}).reset_index()
df_summary.head()